## Imports

In [9]:
import kagglehub
import pandas as pd
import numpy as np
import chardet
from pathlib import Path


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

/opt/anaconda3/envs/DS_Ass2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.29G/2.29G [01:55<00:00, 21.4MB/s]

Extracting model files...


Path to dataset files: /Users/Gabriela/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2


In [10]:
def get_data(file_path: Path) -> pd.DataFrame:
    # We need to detect the encoding first, because it apparently is not UTF-8
    with open(file_path, "rb") as f:
        enc = chardet.detect(f.read(100000))
    return pd.read_csv(file_path, encoding=enc["encoding"])

In [11]:
get_data(path)

IsADirectoryError: [Errno 21] Is a directory: '/Users/Gabriela/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2'